# Prediction of Debt Crisis Events

Inspired by Manassee-Roubini (2009), the purpose of this analysis is to investigate the set of economic and political conditions that are associated with a likely occurrence of a debt crisis. 


## Reference
Manasse, Paolo & Roubini, Nouriel, 2009. ""Rules of thumb" for sovereign debt crises," Journal of International Economics, Elsevier, vol. 78(2), pages 192-205, July.

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import sklearn

In [ ]:
# read in crisis data
crisis = pd.read_excel("crisisdata.xls")
crisis.head()

,Country,year,gfn2gdp,efn2gdp,FCD2GDweo,grossdebt2gdpweo,externdebt2gdp,credit2privat,stdebt2extdebt,CA2gdp,debtserv2res,IntExp2rev,interExtdeb2gdp,debtserv2expor,crisis
0,Albania,1990,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,-2.690549,NaN,NaN,0.000000,NaN,0
1,Albania,1991,NaN,NaN,NaN,NaN,3.955623,NaN,0.0,-4.312943,NaN,NaN,0.139587,2.180682,1
2,Albania,1996,NaN,NaN,NaN,NaN,27.356787,3.832701,0.0,-1.968338,NaN,NaN,0.208382,2.692118,0
3,Albania,1997,NaN,NaN,NaN,NaN,26.352253,3.763376,0.0,-7.893714,NaN,NaN,0.406013,8.060182,1
4,Albania,1999,NaN,NaN,34.046757,0.711837,39.140972,3.837488,0.0,-6.943784,NaN,33.784084,0.605327,9.086977,0


The variable descriptions are as follows: 
- grossdebt2gdpweo: Gross debt (%GDP) 
- efn2gdp: external financing need EFN (% GDP) 
- externdebt2gdp: external debt (%GDP) 
- gfn2gdp: gross financing need GFN (% GDP) 
- interExtdeb2gdp: Interest on external debt (% GDP) 
- CA2gdp: Current account (% GDP) 
- debtserv2expor: Debt service/Exports (%) 
- credit2privat: Credit to private sector (% GDP) 
- FCD2GDweo : Foreign currency denominated debt (% public debt) 
- IntExp2rev: Interest Expenditure/Government revenue (%) 
- debtserv2res: Debt services/Reserves (%) 
- stdebt2extdebt: Short term debt /External debt (%) 

### Part 1: Understanding the Data

#### 1. Present descriptive statistics on the outcome variable. Comment. 

In [ ]:
# descriptive statistics on outcome, crisis

# frequency and proportion 
counts = crisis['crisis'].value_counts()
prop = round(100*counts[1]/sum(counts),5)

print(f'''
             Count   Percentage
Crisis = 0:  {counts[0]}    {100-prop}
Crisis = 1:  {counts[1]}     {prop}
''')


             Count   Proportion
Crisis = 0:  2715    93.42739
Crisis = 1:  191     6.57261



Here we can see the frequency and proportion of the outcome variable, crisis. We note that there is 2715 observations with crisis = 0, and 191 observations with crisis = 1, giving us percentages of 93.42% and 6.57%, respectively. 

This is interesting as we can see that the dataset may be imbalanced. An imbalance occurs when one or more classes have very low proportions in the training data as compared to the other classes.

This can be a problem because typically, the minority class (in our case, debt crisis) is more important and therefore the problem is more sensitive to classification errors for the minority class than the majority class. This occurs in many real life situations such as this one, as well as rare/extreme event prediction.

Now, let us split the data into two subsets, crisis and no crisis, then view the summary statistics by subgroup:

In [ ]:
# split into crisis =0 and crisis =1 check descriptive stats
crisis0 = crisis[crisis['crisis']==0]
crisis0.describe()

,year,gfn2gdp,efn2gdp,FCD2GDweo,grossdebt2gdpweo,externdebt2gdp,credit2privat,stdebt2extdebt,CA2gdp,debtserv2res,IntExp2rev,interExtdeb2gdp,debtserv2expor,crisis
count,2715.000000,1352.000000,1196.000000,994.000000,2239.000000,2011.000000,2377.000000,1526.000000,2490.000000,1331.000000,2434.000000,1600.000000,1511.000000,2715.0
mean,2003.892081,7.997088,16.344866,40.863213,0.488368,68.340746,76.887506,0.011155,-0.102589,114.728702,9.272912,2.464459,30.936303,0.0
std,7.978125,12.214868,34.805275,30.807023,0.343407,107.331112,62.320820,0.031036,11.236503,364.401479,9.426396,8.301201,37.935674,0.0
min,1990.000000,-43.115822,-39.087891,0.000000,0.000000,0.000000,0.000823,0.000000,-242.188065,-21.523167,0.000000,0.000000,0.000000,0.0
25%,1997.000000,1.762889,2.966474,12.673278,0.240723,24.179510,29.875000,0.000000,-4.463280,20.891109,3.061741,0.711043,9.544457,0.0
50%,2004.000000,6.688442,9.224253,38.539671,0.426501,39.925087,55.413284,0.000000,-1.030251,48.610882,6.897833,1.330208,21.195290,0.0
75%,2011.000000,12.367588,19.066620,65.211071,0.655708,69.141159,115.425003,0.006335,3.563094,99.297058,12.094511,2.254286,39.068693,0.0
max,2017.000000,110.963074,498.524750,127.325294,3.426656,1174.712769,421.024994,0.423879,64.969093,6108.248535,111.721611,126.146233,420.474579,0.0


In [ ]:
# crisis = 1
crisis1 = crisis[crisis['crisis']==1]
crisis1.describe()

,year,gfn2gdp,efn2gdp,FCD2GDweo,grossdebt2gdpweo,externdebt2gdp,credit2privat,stdebt2extdebt,CA2gdp,debtserv2res,IntExp2rev,interExtdeb2gdp,debtserv2expor,crisis
count,191.000000,73.000000,76.000000,67.000000,118.000000,176.000000,159.000000,125.000000,175.000000,97.000000,152.000000,155.000000,133.000000,191.0
mean,2000.900524,13.440484,18.299789,61.031404,0.620648,67.356581,40.256836,0.017571,-5.250366,339.898026,14.626044,3.519374,36.234540,1.0
std,8.509162,13.465229,21.344665,23.925811,0.434534,93.685837,46.865476,0.055291,9.133821,1300.707935,13.518846,9.619867,48.025205,0.0
min,1990.000000,-6.123044,-8.445910,0.616486,0.058736,0.000000,1.266927,0.000000,-73.215492,0.839750,0.000000,0.000000,0.000000,1.0
25%,1993.000000,3.965239,5.385578,46.273550,0.301422,29.416216,15.413882,0.000000,-8.502649,30.418505,4.898766,0.707463,10.092102,1.0
50%,1999.000000,9.401822,13.612785,57.873913,0.557577,50.268421,25.672920,0.000000,-3.977028,68.809967,11.035178,1.921634,19.847471,1.0
75%,2009.000000,20.196218,21.988723,82.255817,0.803563,68.053633,48.420927,0.007146,-1.309848,159.432846,20.418385,3.177070,43.539005,1.0
max,2017.000000,61.179409,122.522301,100.000000,2.264065,901.229187,312.117859,0.533167,26.701317,10538.450195,86.565895,86.766556,296.080353,1.0


Here we can see the summary statistics for both groups. As a sanity check, we can look to make sure that the values make sense intuitively, given the context of the question. Here we can see that the values for variables relating to debt are higher when we are in crisis, which makes sense. 

#### 2. Present descriptive statistics on the input variables depending on the default status.

In [ ]:
crisis.describe()

,year,gfn2gdp,efn2gdp,FCD2GDweo,grossdebt2gdpweo,externdebt2gdp,credit2privat,stdebt2extdebt,CA2gdp,debtserv2res,IntExp2rev,interExtdeb2gdp,debtserv2expor,crisis
count,2906.000000,1425.000000,1272.000000,1061.000000,2357.000000,2187.000000,2536.000000,1651.000000,2665.000000,1428.000000,2586.000000,1755.000000,1644.000000,2906.000000
mean,2003.695458,8.275942,16.461670,42.136794,0.494990,68.261545,74.590867,0.011641,-0.440623,130.023818,9.587559,2.557628,31.364932,0.065726
std,8.046797,12.335433,34.147759,30.802815,0.349616,106.278805,62.097167,0.033509,11.182149,490.704999,9.792534,8.428237,38.858821,0.247845
min,1990.000000,-43.115822,-39.087891,0.000000,0.000000,0.000000,0.000823,0.000000,-242.188065,-21.523167,0.000000,0.000000,0.000000,0.000000
25%,1997.000000,1.942951,3.176368,14.569268,0.242832,24.735479,28.180852,0.000000,-4.672428,21.238802,3.125898,0.710925,9.571682,0.000000
50%,2004.000000,6.803134,9.499497,40.013073,0.429259,40.758839,52.932198,0.000000,-1.270393,49.171139,7.089282,1.351801,21.059856,0.000000
75%,2011.000000,12.658605,19.360725,67.044731,0.663545,68.904839,110.318752,0.006373,3.317413,102.077942,12.508049,2.335770,39.379719,0.000000
max,2017.000000,110.963074,498.524750,127.325294,3.426656,1174.712769,421.024994,0.533167,64.969093,10538.450195,111.721611,126.146233,420.474579,1.000000


In [ ]:
crisis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           2906 non-null   object 
 1   year              2906 non-null   int64  
 2   gfn2gdp           1425 non-null   float64
 3   efn2gdp           1272 non-null   float64
 4   FCD2GDweo         1061 non-null   float64
 5   grossdebt2gdpweo  2357 non-null   float64
 6   externdebt2gdp    2187 non-null   float64
 7   credit2privat     2536 non-null   float64
 8   stdebt2extdebt    1651 non-null   float64
 9   CA2gdp            2665 non-null   float64
 10  debtserv2res      1428 non-null   float64
 11  IntExp2rev        2586 non-null   float64
 12  interExtdeb2gdp   1755 non-null   float64
 13  debtserv2expor    1644 non-null   float64
 14  crisis            2906 non-null   int64  
dtypes: float64(12), int64(2), object(1)
memory usage: 340.7+ KB


Using the .describe() function, we can see the summary statistics for the input variables. This function provides the count, mean, std, min, max, and quartiles. Moreover, the .info() function gives us information on the type of each variable, and if there are any missing values. We see here that most of the variables are numeric, with country the only categorical variable. We can deal with missing values and categorical variables next in the data preprocessing step.

### Data Cleaning

Next, I am going to preprocess the data by dealing with missing values and encoding the proper variables. In the next section, I will also standardize the data before passing it into our classifiers. 

#### Missing Values
In order to deal with missing values, I will consider imputing by the process of linear interpolation. 

Linear interpolation is the technique of determining the values of the functions of any intermediate points when the values of two adjacent points are known. In essence, it is the estimation of an unknown value that falls within two known values.

Because this process is forward linear, the first values for each variable will not be imputed. To overcome this, I will continue with mean imputation for the remaining missing values. As the name suggests, mean imputation is a method in which the mean of the observed values for each variable is computed and the missing values for that variable are imputed by this mean.

**Note:** I am dealing with missing values here, however, the process as well as it's drawbacks will be described again in part 5 below.

In [ ]:
# missing values
crisis = crisis.interpolate()
crisis = crisis.fillna(crisis.mean())

<ipython-input-207-f4cb2727f35f>:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  crisis = crisis.fillna(crisis.mean())


In [ ]:
crisis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906 entries, 0 to 2905
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           2906 non-null   object 
 1   year              2906 non-null   int64  
 2   gfn2gdp           2906 non-null   float64
 3   efn2gdp           2906 non-null   float64
 4   FCD2GDweo         2906 non-null   float64
 5   grossdebt2gdpweo  2906 non-null   float64
 6   externdebt2gdp    2906 non-null   float64
 7   credit2privat     2906 non-null   float64
 8   stdebt2extdebt    2906 non-null   float64
 9   CA2gdp            2906 non-null   float64
 10  debtserv2res      2906 non-null   float64
 11  IntExp2rev        2906 non-null   float64
 12  interExtdeb2gdp   2906 non-null   float64
 13  debtserv2expor    2906 non-null   float64
 14  crisis            2906 non-null   int64  
dtypes: float64(12), int64(2), object(1)
memory usage: 340.7+ KB


Here we can see that we now have no missing values. Next, we can encode our categorical variables.

#### Encoding

As discussed in Exercise 2, encoding categorical variables is important as the way we represent our variables can have an effect on the way they interact with our methods used. As we saw in the previous question, we have only one categorical variable in the dataset, namely, Country.

The process I will be using to encode the country variable is one hot encoding. As previously discussed, this method creates a new dummy variable for each level of the variable. Integer encoding is not a good choice here as the integer values have a natural ordered relationship between each other and machine learning algorithms may be able to understand and harness this relationship. In order words, it introduces an ordinal relationship to the variable. Since country is not ordered, it is best that we just create dummy variables. 

In [ ]:
# encode country as categorical using OneHotEncoder
# import function
from sklearn.preprocessing import OneHotEncoder

# define encoder
onehotencoder = OneHotEncoder()

# reformat 1-D country array to 2-D
country_ohe = onehotencoder.fit_transform(crisis.Country.values.reshape(-1,1)).toarray()

# add back into original data frame
crisis_ohe = pd.DataFrame(country_ohe, columns = ["Country_"+str(int(i)) for i in range(country_ohe.shape[1])])
crisis = pd.concat([crisis, crisis_ohe], axis=1)
crisis = crisis.drop(['Country'], axis=1)

# drop one of the dummy variables to avoid multicollinearity
crisis = crisis.drop(['Country_0'], axis=1)

In [ ]:
crisis.head()

,year,gfn2gdp,efn2gdp,FCD2GDweo,grossdebt2gdpweo,externdebt2gdp,credit2privat,stdebt2extdebt,CA2gdp,debtserv2res,...,Country_107,Country_108,Country_109,Country_110,Country_111,Country_112,Country_113,Country_114,Country_115,Country_116
0,1990,8.424674,26.004948,40.327620,0.504628,0.000000,73.924386,0.014401,-2.690549,223.078809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1991,8.424674,26.004948,40.327620,0.504628,3.955623,73.924386,0.000000,-4.312943,223.078809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1996,8.424674,26.004948,40.327620,0.504628,27.356787,3.832701,0.000000,-1.968338,223.078809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1997,8.424674,26.004948,40.327620,0.504628,26.352253,3.763376,0.000000,-7.893714,223.078809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1999,8.424674,26.004948,34.046757,0.711837,39.140972,3.837488,0.000000,-6.943784,223.078809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we can see that we have new binary/dummy variables for each country. 

### Part 2: Classification

#### 3. Why is it important to split the database into training and test sample? Consider a training sample with year<=2009 and a test sample with year>2009.

It is important to split the data into training and testing sets in order to try and avoid problems such as underfitting or overfitting. Under fitting occurs when the data is unable to encapsulate the relations among data. For example, this can occur when we try to fit non-linear data with a linear model. Overfitting occurs when a model learns both the existing relations among data and noise.

After training our model using the training set, we should test it on unseen data from the test set to make sure the model is not just memorizing specific patterns in the training set. We can also estimate how well our model is performing while using new inputs. This is important as we are interested in  different model performance metrics such as accuracy. 

In the context of this question, we split the train/test by year as we are also interested in time trend.

#### Train/Test Split

Here we can consider a training sample with year<=2009 and a test sample with year>2009.

In [ ]:
train = crisis[crisis['year']<=2009]
test = crisis[crisis['year']>2009]

X_train = train.drop('crisis', axis=1)
X_test = test.drop('crisis', axis=1)
y_train = train['crisis']
y_test = test['crisis']

#### Scale Data

Next, we want to scale the data before passing it into our classifiers as it can help the algorithm get trained well and faster. Scaling is a technique to make them closer to each other, i.e., make data points generalized so that the distance between them will be lower. 

For our purposes, we will be using the StandardScaler from sklearn preprocessing. This scaler standardizes features by removing the mean and scaling to unit variance.

The standard score of a sample x is calculated as:
$$
z = \frac{(x - \mu)} {\sigma}
$$

Where $\mu$ is the mean of the training samples, and $\sigma$ is the standard deviation of the training samples.

In [ ]:
# scale the data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# can also scale using MinMaxScaler, this will put values [0,1]
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

#### 4. Explain the procedure that you will be using to tune your model's parameters and find the accuracy of your model.

In order to tune my model's parameters and find the accuracy of my model, I will be using a k-fold cross-validation procedure for hyperparameter tuning and examining AUC scores as a measure of accuracy. 

The grid search provided by GridSearchCV exhaustively generates candidates from a grid of parameter values specified with the param_grid parameter. This grid is provided by the user and has all the values that we want to try combinations from in our search. It runs through all the different parameters that is fed into the parameter grid and produces the best combination of parameters, based on a scoring metric of your choice (accuracy, f1, etc). In our case, we are interested in AUC as our scoring metric. 

The cv parameter also allows us to specify the the number of cv folds for each combination of parameters. Cross-validation is a resampling procedure used to evaluate machine learning models on a limited data sample. The procedure has a single parameter called $k$ that refers to the number of groups that a given data sample is to be split into. As such, the procedure is often called k-fold cross-validation. In k-fold cross validation, the training set is split into $k$ smaller sets. Then following procedure is followed for each of the k “folds”:

- A model is trained using $k-1$ of the folds as training data;
- The resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

Thus, using this technique, we are able to find the best parameters for our classifiers, i.e., tuning the model's parameters. For this assignment, I will proceed with using $k=10$ folds.

Then, to evaluate model accuracy, we will examine the AUC scores. We can set this to be our scoring metric in the grid search as previously mentioned. 

AUC stands for "Area under the ROC Curve." That is, AUC measures the entire two-dimensional area underneath the entire ROC curve. The Receiver Operator Characteristic (ROC) curve is an evaluation metric for binary classification problems. It is a probability curve that plots the true positive rate (TPR) against the false negative rate (FPR) at various threshold values and essentially separates the ‘signal’ from the ‘noise’. 

Note that the TPR is also called sensitivity and tells us what proportion of the positive class got correctly classified and is calculated as follows:

$$
TPR = \frac{TP}{TP+FN}
$$

and FNR tells us what proportion of the positive class got incorrectly classified by the classifier:
$$
FNR = \frac{FN}{TP+FN}
$$

Where TP = true positives, and FN = false negatives.

The AUC then is the measure of the ability of a classifier to distinguish between classes and is used as a summary of the ROC curve. The higher the AUC, the better the performance of the model at distinguishing between the positive and negative classes.

#### 5. Propose a method to deal with missing values

In order to deal with missing values, I will consider imputing by the process of linear interpolation. 

Linear interpolation is the technique of determining the values of the functions of any intermediate points when the values of two adjacent points are known. In essence, it is the estimation of an unknown value that falls within two known values. This can be done by group, i.e, by country. 

Because this process is forward linear, the first values for each variable will not be imputed. To overcome this, I will continue with mean imputation for the remaining missing values. As the name suggests, mean imputation is a method in which the mean of the observed values for each variable is computed and the missing values for that variable are imputed by this mean.

In practice, we would like to try and avoid mean imputation on the full dataset and there are many dangers that come with this process. For example, mean imputation does not preserve the relationships among variables. Moreover, it leads to an underestimate of the standard error. 

There are alternative methods for dealing with missing values. For example, if there is a variable with extreme missing values, we may decide to just drop the whole column. This is not ideal as we will be losing data. This is the same case with deleting rows/observations with missing data. More often than not, missing values can tell a story themselves. Another way to deal with missing values is to mean impute, but then create a new column that indicates whether or not a variable was imputed. There are many types of deterministic imputation methods that we could also explore, including logical imputation, historical (e.g. carry-forward) imputation, ratio and regression imputation. Some of these options may be costly in time and computation. Ultimately, one must be cautious when altering the original dataset.

Please see the "Data Cleaning" section where the missing values were handled for the code.

#### 6. Using default values, construct the following classifiers to predict debt crisis events and summarize the information in a table. The criteria to measure the accuracy is the AUC.
a. Penalized Logistic Classifier \
b. Decision Tree Classifier \
c. Random Forest classifier \
d. Boosting tree classifier \
e. K-NN classifier with K=5 

First, we will construct and train our classifiers using our training data. Then, we will calculate the AUC values and summarize information in a table:

#### Penalized Logistic

In [ ]:
# construct classifiers

# penalized logistic classifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

log = LogisticRegression()

log_fit = log.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = log_fit.predict(X_test)

# Model Accuracy, how often is the classifier correct?
# print("Penalized Logistic Classifier Accuracy:",metrics.accuracy_score(y_test, y_pred))

Penalized Logistic Classifier Accuracy: 0.9534606205250596


#### Decision Tree

In [ ]:
# decision tree classifier
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()

# train decision tree classifer
dt_fit=dt.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = dt_fit.predict(X_test)

# Model Accuracy, how often is the classifier correct?
# print("Decision Tree Accuracy:",metrics.accuracy_score(y_test, y_pred))

Decision Tree Accuracy: 0.8162291169451074


#### Random Forest

In [ ]:
# random forest classifer
from sklearn.ensemble import RandomForestClassifier

rf=RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)

y_pred=rf.predict(X_test)
#print("Random Forest Accuracy:", metrics.accuracy_score(y_test, y_pred))

Random Forest Accuracy: 0.9534606205250596


#### Boosting Tree

In [ ]:
# boosting tree classifier
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier()

#### K-NN

In [ ]:
# K-NN classifier with K=5
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

Now, let us calculate the AUC values and compare the classifiers:

In [ ]:
# look at metrics (AUC)
from sklearn import metrics
from sklearn.metrics import roc_auc_score

log_auc = roc_auc_score(y_test, log_fit.predict_proba(X_test)[:,1])
dt_auc = roc_auc_score(y_test, dt.predict_proba(X_test)[:,1])
rf_auc = roc_auc_score(y_test, rf.predict_proba(X_test)[:,1])
gb_auc = roc_auc_score(y_test, gb.predict_proba(X_test)[:,1])
knn_auc = roc_auc_score(y_test, knn.predict_proba(X_test)[:,1])

In [ ]:
from tabulate import tabulate

# create table data
table_data = [["Penalized Logistic", log_auc], 
        ["Decision Tree", dt_auc], 
        ["Random Forest", rf_auc], 
        ["Boosting Tree", gb_auc],
        ["k-NN", knn_auc]]
  
# define header names
col_names = ["Classifier", "AUC (Default)"]
  
# display table
print(tabulate(table_data, headers=col_names))

Classifier            AUC (Default)
------------------  ---------------
Penalized Logistic         0.647664
Decision Tree              0.590428
Random Forest              0.734655
Boosting Tree              0.733306
k-NN                       0.629671


Here we can see that the Random Forest and the Boosting Tree classifiers performed the best as they have the highest AUC values (both approx 0.73). The classifer that has the lowest AUC value is the Decision Tree, which only had an AUC value of 0.59. 

#### 7. Default parameters are not necessarily the best for our database. I like this example provided by Will Koehrsen on Towardsdatascience "The best way to think about hyperparameters is like the settings of an algorithm that can be adjusted to optimize performance, just as we might turn the knobs of an AM radio to get a clear signal". So let us tune the model.

#### a. From your understanding of the class, identify at least two key tuning parameters for each method of question 6.

The following lists at least two key tuning parameters for each method of Question 6.

Penalized Logistic Classifier
1. Solver (solver)
2. Penalty (penalty)
3. Penalty strength (C)

Decision Tree Classifier 
1. Split criteria (criterion)
2. Strategy used to choose the split at each node (splitter)
3. Maximum depth of tree (max_depth)

Random Forest classifier 
1. Number of trees (n_estimators)
2. Number of random features to sample at each split point (max_features)


Boosting Tree Classifier
1. Learning rate (learning_rate)
2. Number of trees (n_estimators)
3. Number of rows or subset of the data to consider for each tree (subsample)
4. Depth of each tree (max_depth)

K-NN Classifier with K=5
1. Number of neighbours (n_neighbours)
2. Distance metric (metric)
3. Contribution of members (weight)


#### b. Using gridSearchCV, find the best parameters for each of the methods. Note that it will imply that you provide a grid for each parameter over which you will search.

Now, we can use use GridSearchCV to find the best parameters for each of the methods we have been working with. Using the list of tuning parameters from part (a), we can specify a grid for each parameter over which we will search. 

#### Penalized Logistic

In [ ]:
# use gridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# penalized logistic classifier 
model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2','l1']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
log_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc',error_score=0)
log_result = log_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (log_result.best_score_, log_result.best_params_))

Best: 0.752641 using {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}


#### Decision Tree

In [ ]:
# decision tree classifier 

# define models and parameters
model = DecisionTreeClassifier()
criterion = ['gini','entropy']
splitter = ['best','random']
max_depth = [3,7,9]

# define grid search
grid = dict(criterion=criterion,splitter=splitter,max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
dt_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc',error_score=0)
dt_result = dt_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (dt_result.best_score_, dt_result.best_params_))

Best: 0.701494 using {'criterion': 'entropy', 'max_depth': 3, 'splitter': 'best'}


#### Random Forest

In [ ]:
# random forest classifier 
from sklearn.ensemble import RandomForestClassifier

# define models and parameters
model = RandomForestClassifier()
n_estimators = [10, 100, 1000]
max_features = ['sqrt', 'log2']

# define grid search
grid = dict(n_estimators=n_estimators,max_features=max_features)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
rf_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc',error_score=0)
rf_result = rf_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (rf_result.best_score_, rf_result.best_params_))

Best: 0.794525 using {'max_features': 'log2', 'n_estimators': 1000}


#### Boosting Tree

In [ ]:
# boosting tree classifier 
from sklearn.ensemble import GradientBoostingClassifier

# define models and parameters
model = GradientBoostingClassifier()
n_estimators = [10, 100, 1000]
learning_rate = [0.001, 0.01, 0.1]
subsample = [0.5, 0.7, 1.0]
max_depth = [3, 7, 9]

# define grid search
grid = dict(learning_rate=learning_rate, n_estimators=n_estimators, subsample=subsample, max_depth=max_depth)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
gb_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc',error_score=0)
gb_result = gb_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (gb_result.best_score_, gb_result.best_params_))

Best: 0.787957 using {'learning_rate': 0.001, 'max_depth': 9, 'n_estimators': 1000, 'subsample': 0.5}


#### K-NN

In [ ]:
# K-NN classifier with K=5 
from sklearn.neighbors import KNeighborsClassifier

# define models and parameters
model = KNeighborsClassifier()
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']

# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
knn_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='roc_auc',error_score=0)
knn_result = knn_search.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (knn_result.best_score_, knn_result.best_params_))

Best: 0.758475 using {'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'distance'}


#### c. Complete the table in the previous section by comparing the gain in terms of accuracy from the tuning procedure vs. the default parameters procedures.

In [ ]:
# look at metrics (AUC)
log_auc_tuned = log_result.best_score_
dt_auc_tuned = dt_result.best_score_
rf_auc_tuned = rf_result.best_score_
gb_auc_tuned = gb_result.best_score_
knn_auc_tuned = knn_result.best_score_

In [ ]:
# create table data
table2_data = [["Penalized Logistic", log_auc, log_auc_tuned, log_auc_tuned-log_auc], 
        ["Decision Tree", dt_auc, dt_auc_tuned, dt_auc_tuned-dt_auc], 
        ["Random Forest", rf_auc, rf_auc_tuned, rf_auc_tuned-rf_auc], 
        ["Boosting Tree", gb_auc, gb_auc_tuned, gb_auc_tuned-gb_auc],
        ["k-NN", knn_auc, knn_auc_tuned, knn_auc_tuned-knn_auc]]
  
# define header names
col_names2 = ["Classifier", "AUC (Default)", "AUC (Tuned)", "Difference"]
  
# display table
print(tabulate(table2_data, headers=col_names2))

Classifier            AUC (Default)    AUC (Tuned)    Difference
------------------  ---------------  -------------  ------------
Penalized Logistic         0.647664       0.752641     0.104977
Decision Tree              0.590428       0.701494     0.111066
Random Forest              0.734655       0.794525     0.0598702
Boosting Tree              0.733306       0.787957     0.0546507
k-NN                       0.629671       0.758475     0.128804


Here we can see a table with the AUC values for all of the classifiers for both the default and the tuned models. As we previously saw, the Random Forest and the Boosting Tree classifiers had the highest AUC values with the default parameters. After tuning the models, we see that they still have the highest values. However, we can note that there seemed to be significant differences in the AUC of the default and the tuned models for the other classifiers, namely the Penalized Logistic, Decision Tree, and k-NN. For these classifiers, the difference in AUC between the default and the tuned models was between 0.10-0.13. This is quite the difference contrasted to the differences of approx 0.5-0.6 that the Random Forest and Boosting Tree classifiers saw.

We note that the Random Forest and Boosting Tree classifiers had good AUCs to begin with, and the GridSearch is costly in time. For the other classifiers, it is worth it as we see great improvements. Now, all of the tuned AUC values are in the 0.75-0.79 range, which is pretty good for a classifier. This indicates that the models are performing working well.  

#### 8. Other variables could matter for the prediction. I am thinking in particular about growth variables. Compute the growth of all the variables in the database (you can help yourself by using Stata at this stage or Matlab or any other software with which you feel more comfortable calculating growth rates).

Growth rates for this question will be calculated in R. The code will be available in the .rmd file. 

To calculate growth rate, we will start by subtracting the past value from the current value. Then, we divide that number by the past value. Finally, we can multiply our answer by 100 to express it as a percentage. It is important to note that these growth rates must be calculated by country and year. 

Let us export our dataset with imputed values to read into R.

In [ ]:
crisis.to_csv("crisis_ohe_imputed.csv")

#### 9. Using the most correlated variables (similar to what we have seen in Ferrara-Simoni 2019), select the variable that will enter your model and do the prediction using a lasso-logit. Note that you should consider two tuning parameters in this case: the degree of the penalty and the number of correlated variables to consider. Why might selecting the most correlated variables matter?

Now, let us read in our new dataset with calculated growth rates from question 8.

In [ ]:
# read in new data with growth rates
crisis2 = pd.read_csv("crisis_growth.csv")
crisis2 = crisis2.fillna(crisis2.mean())

# look at correlations with the outcome variable
corr=pd.DataFrame(crisis2[crisis2.columns[1:]].corr()['crisis'][:])
corr['abs_corr'] = corr['crisis'].abs()
corr_sorted = corr.sort_values(by=['abs_corr'],ascending=False)

corr_sorted.head(50)

,crisis,abs_corr
crisis,1.000000,1.000000
credit2privat,-0.132380,0.132380
int_exp2rev,0.121315,0.121315
fcd2g_dweo,0.109843,0.109843
country_91,0.107120,0.107120
ca2gdp,-0.106505,0.106505
year,-0.092141,0.092141
gfn2gdp,0.090052,0.090052
grossdebt2gdpweo,0.084956,0.084956
country_4,0.077788,0.077788


We are interested in selecting only the variables that are highly correlated with our outcome variable, crisis. Looking at the output above, we see that the top 5 most correlated variables are credit2privat, int_exp2rev, fcd2g_dweo, ca2gdp, and year. This is interesting as none of the growth variables we calculated appeared near the top, the closest one being debtserv2expor_growth with an absolute correlation of 0.017776. 

Note that the country variables appear near the top as well. When dealing with categorical variables and model selection, we should not only include one 
level of the variable and not the others. Either we include all levels, or none. It is important to treat all of these dummy variables as a whole. In essence, by only including one level of the categorical variable, we can now changed the original variable, as we will no longer be able to refer back to the baseline. Because of this, I will not include single country variables.

Let us first try the model with the top 5 correlated variables.

In [ ]:
# prediction using LASSO logit
train2 = crisis2[crisis2['year']<=2009]
test2 = crisis2[crisis2['year']>2009]

X_train5 = train2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp', 'year']]
X_test5 = test2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp', 'year']]

# scale the data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train5 = scaler.fit_transform(X_train5)
X_test5 = scaler.transform(X_test5)

log5 = LogisticRegression(penalty='l1', solver='liblinear')
log5.fit(X_train5, y_train)

# look at AUC
from sklearn import metrics
log_auc5 = roc_auc_score(y_test, log5.predict_proba(X_test5)[:,1])
log_auc5

0.6982236842105264

Now, let us try with the top 7 correlated variables.

In [ ]:
X_train7 = train2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp','year','gfn2gdp','grossdebt2gdpweo']]
X_test7 = test2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp', 'year','gfn2gdp','grossdebt2gdpweo']]

# scale the data 
scaler = StandardScaler()
X_train7 = scaler.fit_transform(X_train7)
X_test7 = scaler.transform(X_test7)

log7 = LogisticRegression(penalty='l1', solver='liblinear')
log7.fit(X_train7, y_train)

# look at AUC
log_auc7 = roc_auc_score(y_test, log7.predict_proba(X_test7)[:,1])
log_auc7

0.7084210526315788

Finally, we can try with the top 10 correlated variables.

In [ ]:
X_train10 = train2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp','year','gfn2gdp','grossdebt2gdpweo','inter_extdeb2gdp','debtserv2expor','efn2gdp']]
X_test10 = test2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp', 'year','gfn2gdp','grossdebt2gdpweo','inter_extdeb2gdp','debtserv2expor','efn2gdp']]

# scale the data 
scaler = StandardScaler()
X_train10 = scaler.fit_transform(X_train10)
X_test10 = scaler.transform(X_test10)

log10 = LogisticRegression(penalty='l1', solver='liblinear')
log10.fit(X_train10, y_train)

# look at AUC
log_auc10 = roc_auc_score(y_test, log10.predict_proba(X_test10)[:,1])
log_auc10

0.6995394736842105

Here we can see that the model with the top 7 correlated variables has the highest AUC value of 0.708. Thus, we can keep the model with the top 7 correlated variables.

Selecting the most correlated variables is important as we are interested in finding the variables that best predict the outcomes. One issue with the approach taken here (looking soley at correlation coefficients), is that issues like multicollinearity may arise. An alternative method would be to run a series of regressions, regressing each individual independent variable on the outcome variable, crisis, and then examining the t-statistics/F-scores. By looking at the significant results, we can then choose the predictors to enter our model. Additionally, there are other tests that can be used when trying to compare models such as the likelihood ratio test (LRT) and residual deviance tests.

In general, just looking at correlated variables as a way of model selection is not always sufficient. One should be careful when using different techniques for model selection. Other methods include automatic selection procedures such as forward/backward subset selection. When selecting a model, these procedures and techniques should be used with caution and in combination with domain specific knowledge. Moreover, one must keep in mind if they are more interested in using the model for inference or prediction. 

#### 10. Because the cost of missing a crisis is high, governments worry more about it. What does this mean for your accuracy metric? Redo the setting from question 9. Comment.

Because the cost of missing a crisis is high, we want to make sure to try and minimize false negatives. A false negative is an outcome where the model incorrectly predicts the negative class. In our context, a false negative is when the model predicts no crisis, when in reality, there is one. 

Recall the concepts of type I and II errors. A type I error occurs when the null hypothesisis true, but is rejected (false positive). A type II error occurs when the null hypothesis is false, but erroneously fails to be rejected (false negative).

With this in mind, a metric that we can try to maximize then is recall. Recall is the number of true positives divided by the total number of elements that actually belong to the positive class. This is also known as sensitivity or the TPR, as previously seen in question 4.

$$
\text{Recall} = \frac{TP}{TP+FN}
$$

More generally, recall is simply the complement of the type II error rate, i.e. one minus the type II error rate. Thus, to maximize recall, we must minimize type II error, and in turn minimize our false negatives.

Therefore, we can redo the setting from question 9, but this time using recall as our accuracy metric. Let us do some hyperparameter tuning to find the optimal model.

In [ ]:
# redo question 9, using recall as accuracy metric 

# use top 5 correlated variables
X_train7 = train2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp','year','gfn2gdp','grossdebt2gdpweo']]
X_test7 = test2[['credit2privat', 'int_exp2rev', 'fcd2g_dweo','ca2gdp', 'year','gfn2gdp','grossdebt2gdpweo']]

# scale the data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train7 = scaler.fit_transform(X_train7)
X_test7 = scaler.transform(X_test7)

log7 = LogisticRegression(penalty='l1', solver='liblinear')
log7.fit(X_train7, y_train)

# use gridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer

# penalized logistic classifier 
model = LogisticRegression(penalty='l1')
solvers = ['liblinear'] #note that newton-cg and lbfgs only supports l2 penalty or none
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10,  n_repeats=3, random_state=1)
log7_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring = 'recall',error_score=0)
log7_result = log_search.fit(X_train7, y_train)

# summarize results
print("Best: %f using %s" % (log7_result.best_score_, log7_result.best_params_))

Best: 0.000000 using {'C': 100, 'solver': 'liblinear'}


Here we see that the best parameters are C = 100, and the liblinear solver. Using recall as our accuracy measure, we find that we get a very poor/low recall value. Intuitively, I suppose that this is due to the fact that the dataset is very imbalanced towards the zero class, thus maximizing the recall puts all the predictions there since it has a lot more samples. I discuss other ways to further improve the accuracy of the prediction in the next question.

#### 11. How could you further improve the accuracy of the prediction?

As with any analysis, it is always good to consider how we can improve our model. In order to further improve the accuracy of the prediction, there are many steps we can take all throughout the model development process. Here I will name a few:

**Adding more data**
- If possible, we can try to collect more data points. This will allow the data to "speak for itself", so to speak. Having more data in our dataset will help the model train more accurately by capturing a larger portion of the data distribution.
- Moreover, there may be different variables that we can add to our model that we do not yet have.  
    
**Deal with missing data differently** 
- The presence of missing values can reduce the accuracy of the model or lead to a biased model and inaccurate predictions. This is because we don’t analyze the behavior and relationship with other variables correctly.
- As mentioned in question 5, there are alternative ways of dealing with missing values. It could be of value to try out different, more complex/precise ways of dealing with missing values instead of the way that was done here. 
- For example, different types of deterministic imputation methods include logical imputation, historical (e.g. carry-forward) imputation, mean imputation, ratio and regression imputation. We could also try imputation with an additional column to signifiy if a value has been imputed or not.  

**Treat outliers**
- Another thing we can do is try and deal with outliers in the data.
- We can delete the observations, perform transformation, binning, impute, or treat outlier values separately.

**Derive and transform variables**
- This step can also be thought of as feature engineering.
- We can try and see if there are any derived variables we can create using the data we have. This usually goes with domain knowledge, as well as thinking about the original research question.
- Transforming variables such as scaling can help as well. In this exercise, we scaled using standardization. We can also try the MinMaxScaler() in a future analysis to put the data point on the (0,1) interval.  

**Model selection**
- We can use different model selection techniques to try and find the best predictors to add to our model. 
- We can use principal components analysis, which is a dimensionality reduction technique that helps to represent training data into lower dimensional spaces, but still characterize the inherent relationships in the data.
- As discussed briefly in question 9,  an alternative method would be to run a series of regressions, regressing each individual independent variable on the outcome variable, crisis, and then examining the t-statistics/F-scores. We could also use automatic selection procedures such as backwards/forwards subset selection, or use statistical tests like LRT or residual deviance to compare models. 
- Again, it is crucial to keep in mind that when selecting a model, these procedures and techniques should be used with caution and in combination with domain specific knowledge.  

**Hyperparameter tuning**
- We can also try to tune more hyperparamters. Some classiers we can tune more hyperparameters, as we saw in question 7a, there are lots of different parameters we can tune, some classifiers more than others.
- Moreover, we can also try using different other types of classifiers.  

#### 12. Summarize your findings.

**Summary**  
For this analysis, we first started with our exploratory data analysis (EDA) by looking at discriptive statistics of our variable of interests. After that, we did some data preprocessing which involved things such as dealing with missing values, variable endcoding, and standardization. Then, we split the data into training and testing sets to be used to train and evaluate our model. 

We used 5 different classifiers, namely, penalized logistic, decision tree classifier, random forest classifier, boosting tree, and K-NN classifier (with K=5). We first fit the models with the default parameters and found that the random forest and the boosting tree performed the best using AUC as our ccuracy metric. We then repeated the analysis, this time using GridSearchCV to tune our hyperparameters. Again, we found that the random forest and boosting tree had the highest AUC, but we saw considerable improvement in the other classifiers. 

Next, we calculated growth rates and reviewed the most correlated variables. Ultimately, we selected the top 7 correlated variables, i.e., credit2privat, int_exp2rev, fcd2g_dweo, ca2gdp, year, gfn2gdp, and grossdebt2gdpweo. We then repeated the exercise using recall as our accuracy metric as we wanted to minimize false negatives (missing a crisis), and to do so, we wanted to minimize type II errors (maximize recall). For this analysis, we had a very bad value of recall, which could be attributed to our inbalanced dataset. Further work can be done to improve upon this analysis as we discussed in question 11 above. 

**Reflection**  
Now, I will reflect on the analysis and discuss what went well, and what didn't. There are a few key take-aways that I got from this modelling exercise, namely:

- In practice, dealing with missing values is not easy! The decisions you make on how to treat such values can greatly affect how the algorithm will perform and how we represent the data. More generally, data preparation is an intensive task that is very important, and must be done before we even get to start modelling.
- Modelling, and machine learning, is an art. There are usually multiple ways of doing things right, and given a set to analyze, each individual's path will not be same. Hyperparamter tuning is an iterative process, and we should be mindful of all the decisions we make and what their assumptions/implications are in every step of the model development process. 
- Things should always be taken in context. We must be mindful of the data we are working with, what our goal is, how was this data collected, how does this relate to our hypothesis, etc. 